# Jeremy notebook for final project

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import k_means
from helpers import *
from sklearn import cluster
%load_ext autoreload
%autoreload 2

## Table segmentation

In [7]:
def process_image(image) -> dict:
    """
    Process image and return information. To return the value of the cards we use 
    the following format: {number}{color}. Where 
        - color is either (D)imanond, (H)eart, (S)pade, (C)lub
        - number is either 2-10, (J)ack, (Q)ueen, (K)ing. A(ce).
            
    Parameters
    ----------
    image:
        Input image to process

    Returns
    -------
    results:
        - T1 (str): String code of the first card of the flop (far left) 
        - T2 (str): String code of the second card of the flop
        - T3 (str): String code of the third card of the flop
        - T4 (str): String code of the turn
        - T5 (str): String code of the river
        - P11 (str or int): String code of player 1 card 1. Should be 0 if player 1 is not Playing.
        - P12 (str or int): String code of player 1 card 2. Should be 0 if player 1 is not Playing.
        - P21 (str or int): String code of player 2 card 1. Should be 0 if player 2 is not Playing.
        - P22 (str or int): String code of player 2 card 2. Should be 0 if player 2 is not Playing.
        - P31 (str or int): String code of player 3 card 1. Should be 0 if player 3 is not Playing.
        - P32 (str or int): String code of player 3 card 2. Should be 0 if player 3 is not Playing.
        - P41 (str or int): String code of player 4 card 1. Should be 0 if player 4 is not Playing.
        - P42 (str or int): String code of player 4 card 2. Should be 0 if player 4 is not Playing.
        - CR (int): Number of (r)ed chips in the pot
        - CG (int): Number of (g)reen chips in the pot
        - CB (int): Number of (b)lue chips in the pot
        - CK (int): Number of blac(k) chips in the pot
        - CW (int): Number of (w)hite chips in the pot
        
    Examples
    --------
    - QH: Queen of hearths
    - 10S: 10 of spades
    - AC: Ace f clubs

    """
    
    table_segmentation = segment_table(image)

    # Pre process
    kernel = np.ones((5,5),np.float32)/25
    table_filtered = cv2.filter2D(table_segmentation,-1,kernel)
    table_canny = cv2.Canny(table_filtered, 10, 200)

    # Extract cards
    T_cards = extract_T_cards(table_segmentation, table_canny)
    right_cards = extract_right_cards(table_segmentation, table_canny)
    left_cards = extract_left_cards(table_segmentation, table_canny)
    top_cards = extract_top_cards(table_segmentation, table_canny)

    # Split up into explicit parts
    T1, T2, T3, T4, T5 = T_cards[0], T_cards[1], T_cards[2], T_cards[3], T_cards[4]
    P1 = right_cards[0]
    P2 = top_cards[0]
    P3 = top_cards[1]
    P4 = left_cards[0]
    
    # Classify cards here
    #classification = classify([T1, T2, T3, T4, T5, P1, P2, P3, P4])

    #T1, T2, T3, T4, T5 = classification[0], classification[1], classification[2], classification[3], classification[4]
    #P1 = classification[5]
    #P2 = classification[6]
    #P3 = classification[7]
    #P4 = classification[8]
    #P11 = P1[0]
    #P12 = P1[1]
    #P21 = P2[0]
    #P22 = P2[1]
    #P31 = P3[0]
    #P32 = P3[1]
    #P41 = P4[0]
    #P42 = P4[1]
    
    # Count chips
    labels = get_chips_labels(table_segmentation, plot=False)
    num_white_chips = (labels==0).sum()
    num_black_chips = (labels==1).sum()
    num_red_chips = (labels==2).sum()
    num_green_chips = (labels==3).sum()
    num_blue_chips = (labels==4).sum()
    
    results = {
        # Flop, river and turn
        "T1": "T1", "T2": "T2", "T3": "T3", "T4": "T4", "T5": "T5",
        # Player cards
        "P11": 'P11' , "P12": 'P12', "P21": 'P21', "P22": "P22", "P31": "P31", "P32":"P32" , "P41": "P41", "P42": "P42",
        # Chips
         "CR": num_red_chips, "CG": num_green_chips, "CB": num_blue_chips,"CK": num_black_chips, "CW": num_white_chips
    }
    return results



In [8]:
image = cv2.imread("data/train/train_06.jpg")
table_result = process_image(image)
table_result

{'T1': '2C',
 'T2': 'AH',
 'T3': '8D',
 'T4': 'JC',
 'T5': 'JS',
 'P11': '2H',
 'P12': '3C',
 'P21': 'KS',
 'P22': 'QS',
 'P31': 'KD',
 'P32': '5S',
 'P41': '7H',
 'P42': '8H',
 'CR': 0,
 'CG': 0,
 'CB': 1,
 'CK': 0,
 'CW': 0}

In [3]:
plt.imshow(image)

NameError: name 'plt' is not defined